In [209]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm
from torchsummary import summary
from TCN import TemporalConvNet
import argparse
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset ,DataLoader
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import wandb
import matplotlib.pyplot as plt
import pickle

In [275]:
##train setting
DEVICE = 'cuda:0'
IR = 0.1
EPOCH =400
DROPOUT=0.3
PATIENCE =20
BATCH_SIZE = 1024
NUM_LAYERS =2
SUFFLE = True

input_size = 5
num_channels =[80]*3
kenel_size = 7
output_size= 30
dropout = 0.25


WINDOW_SIZE  = 90
PREDICT_SIZE = 15
SLIDING_SIZE =1
FEATURE_SIZE = 5
BATCH_SIZE =64
FULLLY_SIZE =128
FEATURE = [ 'Open', 'High','Low','Close', 'Volume']
TARGET = 'Close'
TICKER_NUMBER =2743
TOTAL_DAY = 166
WINDOW_NUMBER =int(TOTAL_DAY - (WINDOW_SIZE+PREDICT_SIZE)/SLIDING_SIZE+1)

In [276]:
class TCNModel(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCNModel, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        self.linear = nn.Linear(num_channels[-1], output_size)
        self.init_weights()

    def init_weights(self):
        self.linear.weight.data.normal_(0, 0.01)

    def forward(self, x):
        y1 = self.tcn(x)
        return self.linear(y1[:, :, -1])

In [284]:


data= pd.read_csv('../../../data/stockPrice/NASDAQ.csv')
data.columns = ['Date', 'Ticker', 'Open', 'High','Low', 'Close', 'Volume','sell','buy' ]



def min_max(sequences):
  results = sequences.copy()
  v_min =results.min()
  v_max =results.max()
  new_min =0
  new_max =1
  min_max=[]
  for index,sequence in enumerate(results):
    v_min =sequence.min()
    v_max =sequence.max()
    v_p = (sequence - v_min)/(v_max - v_min)*(new_max - new_min) + new_min
    min_max.append([v_min,v_max])
    results[index] = v_p
  return results, min_max

def inverse_min_max(sequences,min_max):
  original = []
  for index,sequence in enumerate(sequences):
    v_min =min_max[index][0]
    v_max =min_max[index][1]
    sequence_restored = (sequence) * (v_max - v_min) + v_min
    original.append(sequence_restored)
  return np.array(original)

def preprocess_lstm(df):
  sequences = list()
  for group in df.groupby('Ticker'):
    sequences.append(group[1][FEATURE])
  sequences=np.array(sequences)

  ## min_max trading
  price , trade,sentiments =np.split(sequences,[4,5],axis=2)
  trade , min_max_trading =min_max(trade)
  price , min_max_list =min_max(price)
  combine =np.concatenate([price,trade,sentiments],axis =2)

  ## min_max another

  ## min_max another
  result_list= []
  for i in range(0,WINDOW_NUMBER):
    a, b, c= np.split(combine,[i,i+WINDOW_SIZE+PREDICT_SIZE],axis=1)
    result_list.append(b)
  result_array = np.array(result_list)
  print(166//(WINDOW_SIZE+PREDICT_SIZE))
  train ,vaild = np.split(result_array,[166%(WINDOW_SIZE+PREDICT_SIZE)],axis=0)
  train_x,train_y=np.split(train.reshape(166%(WINDOW_SIZE+PREDICT_SIZE)*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)
  vaild_x,vaild_y=np.split(vaild.reshape(1*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)
  return train_x,train_y,vaild_x,vaild_y,min_max_list



class BaseDataset(Dataset):
    def __init__(self, x_data, y_data, target):
        self.x_data = torch.tensor(x_data,dtype =torch.float32)
        self.y_data = torch.tensor(y_data[:,FEATURE.index(target),:],dtype = torch.float32)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.x_data.shape[0]
train_x,train_y,vaild_x,vaild_y,min_max_list = preprocess_lstm(data)

def swap_axes(arr):
    return np.swapaxes(arr, 1, 2)
train_x = swap_axes(train_x)
train_y = swap_axes(train_y)
vaild_x = swap_axes(vaild_x)
vaild_y = swap_axes(vaild_y)
train_x.shape,train_y.shape,vaild_x.shape,vaild_y.shape,len(min_max_list)

train_data = BaseDataset(train_x,train_y,TARGET)
test_data =  BaseDataset(vaild_x,vaild_y,TARGET)
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,drop_last=True)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True,drop_last=True)





1


In [286]:
train_y.shape

(167323, 5, 15)

In [ ]:
train_x.reshape(126178,5,90).shape

(126178, 5, 90)

In [287]:
class TCN_trainer:
    def __init__(self,model, train_loader, test_loader,num_epochs = None, lr = None,batch_size=BATCH_SIZE, verbose = 1, patience=None):
        self.model = model
        self.batch_size = batch_size
        self.train_loader = train_loader
        self.test_loader =test_loader
        self.verbose = verbose
        self.patience =patience
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.criterion = nn.MSELoss().to(self.device)
        self.optimizer =  optim.Adam(model.parameters(), lr = lr)
        self.scheduler =   optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min')
        self.nb_epochs = num_epochs
        self.train_hist  = np.zeros(self.nb_epochs)
        self.vaild_hist =[]

    def train(self):
        config = {
        "model":'TCN',
        "learning_rate": IR,
        "epochs": EPOCH,
        "batch_size": BATCH_SIZE,
        "shuffle":SUFFLE,
        'verbose':1,
        'patience':PATIENCE,
        'dropout':DROPOUT,
        'target':TARGET,
        'feature_columns':FEATURE,
        "NUM_CHANNELS":num_channels,
        'KERNEL':kenel_size,

        }
        run = wandb.init(project ="LSTM",config=config)

        self.model.to(self.device)
        for epoch in range(self.nb_epochs):
            avg_cost = 0
            total_batch = len(self.train_loader)
            for batch_idx, samples in enumerate(self.train_loader):
                x_train, y_train = samples
                x_train =x_train.cuda()
                y_train =y_train.cuda()
                outputs = self.model(x_train)
                loss = self.criterion(outputs, y_train)
                self.optimizer.zero_grad()
                ##loss
                loss.backward()
                self.optimizer.step()
                avg_cost += loss/total_batch
                ##정확도 계산
                predict, label = outputs.clone().detach(),y_train.clone().detach()
                predict = inverse_min_max(predict.to('cpu').numpy(),min_max_list)
                label =  inverse_min_max(label.to('cpu').numpy(),min_max_list)
                score = self.MAE(predict,label)
                acc = self.accuracy(label,predict)
                wandb.log({"Training Loss": loss.item()})
            self.train_hist[epoch] = avg_cost
            if epoch % self.verbose == 0:
                total_loss, score,acc =self.vaild()
                print('Epoch:', '%04d' % (epoch), 'train loss :', '{:.4f}'.format(avg_cost))
                print('vaild_loss:', '{:.4f}'.format (total_loss), 'MAE :', '{:.4f}'.format(score),'Accuarcy :', '{:.4f}'.format(acc))
                wandb.log({"Evaluation Loss": loss.item()})
                wandb.log({"Evaluation MAE": score.item()})
                wandb.log({"Evaluation Accuracy": acc.item()})
                self.vaild_hist.append({"vaild_loss":total_loss ,"vaild_score":score,"vaild_accuracy":acc,"epoch":epoch})
                self.scheduler.step(total_loss)
        # patience번째 마다 early stopping 여부 확인
            if (epoch % self.patience == 0) & (epoch != 0):
                # loss가 커졌다면 early stop
                index =int(epoch/self.patience)
                if index> 1:
                    print((self.vaild_hist[index-1]['vaild_loss'] ,  self.vaild_hist[index]['vaild_loss'])   )
                    if self.vaild_hist[index-1]['vaild_loss']<self.vaild_hist[index]['vaild_loss']:
                        print('\n Early Stopping')
                        break
                    else:
                        print('model was saved')
                        torch.save(self.model,"best-model.pt")
    def vaild(self):
        self.model.to(self.device)
        self.model.eval()
        with torch.no_grad():
            pred = []
            avg_cost = 0
            score = 0
            acc = 0
            total_batch= len(test_dataloader)
            for batch_idx, samples in enumerate(self.test_loader):
                x_test, y_test = samples
                x_test =x_test.cuda()
                y_test =y_test.cuda()
                outputs = self.model(x_test)
                loss = self.criterion(outputs, y_test)
                avg_cost += loss
                predict = inverse_min_max(outputs.to('cpu'),min_max_list)
                y_test =  inverse_min_max(y_test.to('cpu'),min_max_list)
                score += self.MAE(predict,y_test)
                acc += self.accuracy(y_test,predict)
        total_loss = avg_cost/total_batch
        score = score/len(self.test_loader)
        acc = acc/len(self.test_loader)
        self.model.train()
        return total_loss,score,acc

    def MAE(self,true, pred):
         return np.mean(np.abs(true-pred))
    def accuracy(self,true, pred):
        return (1-np.mean(np.abs((true-pred)/true)))




In [288]:
model = TCNModel(input_size=5 ,num_channels=num_channels,output_size=PREDICT_SIZE,kernel_size=kenel_size, dropout=0.25)
trainer = TCN_trainer(model,train_dataloader,test_dataloader,num_epochs=EPOCH,lr= IR,batch_size=BATCH_SIZE,patience=PATIENCE)


/home/ssu36/anaconda3/envs/tiger/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [290]:
trainer.train()
wandb.finish()

Epoch: 0000 train loss : 118108.3438
vaild_loss: 0.0244 MAE : 2.3470 Accuarcy : 0.8917
Epoch: 0001 train loss : 0.0532
vaild_loss: 0.0259 MAE : 2.3142 Accuarcy : 0.8713
Epoch: 0002 train loss : 0.0259
vaild_loss: 0.0202 MAE : 2.1097 Accuarcy : 0.8754


KeyboardInterrupt: 

In [289]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [ ]:
X , y = torch.Tensor(),torch.Tensor()
for batch_idx, samples in enumerate(train_dataloader):
    x_train, y_train = samples
    X =x_train.cuda()
    y =y_train.cuda()
    break



In [252]:
output =model(X)

In [253]:
output.shape

torch.Size([64, 30])